# 1. Загрузка данных, описание поставленной задачи и её первичный анализ

In [ ]:
import numpy as np              # Одномерные и многомерные массивы (array)
import pandas as pd             # Таблицы и временные ряды (dataframe, series)
import matplotlib.pyplot as plt # Научная графика
import seaborn as sns           # Ещё больше красивой графики для визуализации данных
import sklearn                  # Алгоритмы машинного обучения
from sklearn.impute import SimpleImputer # Для заполнения пропущенных значений
from sklearn.model_selection import train_test_split

In [ ]:
from ucimlrepo import fetch_ucirepo
predict_students_dropout_and_academic_success = fetch_ucirepo(id=697)

X = predict_students_dropout_and_academic_success.data.features.copy()
del X['Curricular units 1st sem (credited)']
del X['Curricular units 1st sem (enrolled)']
del X['Curricular units 1st sem (evaluations)']
del X['Curricular units 1st sem (approved)']
del X['Curricular units 1st sem (without evaluations)']
del X['Curricular units 2nd sem (credited)']
del X['Curricular units 2nd sem (enrolled)']
del X['Curricular units 2nd sem (evaluations)']
del X['Curricular units 2nd sem (approved)']
del X['Curricular units 2nd sem (without evaluations)']
y = predict_students_dropout_and_academic_success.data.targets.copy()

X = X.join(y)

In [ ]:
X.shape # Размерность датасета (4424 строки и 27 столбцов-признаков (включая целевой))

In [ ]:
pd.options.display.max_columns = 27
X # Наши исходные ("сырые") данные

Всего студентов, окончивших обучение с дипломом, 2209; отчисленных – 1421;  
только что зачисленных (возможно, уже не в первый раз!) – 794.

In [ ]:
X['Target'].value_counts()

## Постановка задачи
Данная задача является задачей классификации студентов с 3-мя классами:
- **Graduate** (стал выпускником)
- **Enrolled** (только что зачислен)
- **Dropout** (был отчислен за неуспеваемость).

## Описание признаков, используемых в задаче:
- **Marital Status** – семейное положение
- **Application mode** – "категория при зачислении" (обычный студент, иностранный студент и другое)
- **Application order** – какой приоритет данный университет имел для студента при поступлении (от 0 до 9, 0 – студент хотел поступить именно сюда в первую очередь, 9 – студент хотел поступить сюда в последнюю очередь)
- **Course** – специальность (33 – Технологии производства биотоплива 171 – Анимация и мультимедийный дизайн, ...)
- **Daytime/evening attendance** – форма обучения (1 – дневная, 0 – вечерняя)
- **Previous qualification** – предыдущее образование (ступень)
- **Grade of previous qualification** (от 0 до 200) – оценка, полученная за предыдущее образование
- **Nacionality** – национальность
- **Mother's qualification** – уровень образования матери
- **Father's qualification** – уровень образования отца
- **Mother's occupation** – область занятости матери (вид работы)
- **Father's occupation** – область занятости отца (вид работы)
- **Admission grade** – общий балл при поступлении (от 0 до 200)
- **Displaced** – переселенец (1) или нет (0)
- **Educational special needs** – имеет особые образовательные потребности (1 – да, 0 – нет)
- **Debtor** – имеет задолженность оплаты за учёбу (на момент сбора статистики) (1 – да, 0 – нет)
- **Tuition fees up to date** – оплатил учёбу на данный момент (на момент сбора статистики) (1 – да, 0 – нет)
- **Gender** – пол (1 – мужской, 0 – женский)
- **Scholarship holder** – является стипендиатом (1 – да, 0 – нет)
- **Age at enrollment** – возраст на момент зачисления в университет
- **International** – международный (1) или нет (0) студент
- **Curricular units 1st sem (grade)** – средний балл за 1-й семестр обучения (от 0 до 20)
- **Curricular units 2nd sem (grade)** – средний балл за 2-й семестр обучения (от 0 до 20)
- **Unemployment rate** – уровень безработицы в стране обучения (в %)
- **Inflation rate** – уровень инфляции в стране обучения (в %)
- **GDP** – ВВП страны обучения.

Мы будем работать с пересчисленными выше 26 признаками.  
Некоторые побочные признаки были отсечены в самом начале работы ввиду сложнопереводимости их смысла на русский язык.

В нашем датасете все признаки представлены числовыми значениями, но это вовсе не означает, что все они – числовые.  
Многие из них – *числовые (дискретные)*, например, возраст на момент поступления.  
Некоторые – *числовые (непрерывные)*, например, средний балл студента.  
Также среди признаков довольно много *категориальных (неупорядоченных, т.е. номинальных)* и *бинарных*.  
Бинарные (в теории) можно считать как *категориальными неупорядоченными (номинальными)*, так и *числовыми (дискретными)*, но считать, например, что студент одного пола по определению лучше студента другого, было бы странно.

In [ ]:
# Настройка типов признаков в Pandas dataframe
X['Marital Status'] = X['Marital Status'].astype('category')
X['Application mode'] = X['Application mode'].astype('category')
X['Course'] = X['Course'].astype('category')
X['Daytime/evening attendance'] = X['Daytime/evening attendance'].astype('category')
X['Previous qualification'] = X['Previous qualification'].astype('category')
X['Nacionality'] = X['Nacionality'].astype('category')
X['Mother\'s qualification'] = X['Mother\'s qualification'].astype('category')
X['Father\'s qualification'] = X['Father\'s qualification'].astype('category')
X['Mother\'s occupation'] = X['Mother\'s occupation'].astype('category')
X['Father\'s occupation'] = X['Father\'s occupation'].astype('category')
X['Displaced'] = X['Displaced'].astype('category')
X['Educational special needs'] = X['Educational special needs'].astype('category')
X['Debtor'] = X['Debtor'].astype('category')
X['Tuition fees up to date'] = X['Tuition fees up to date'].astype('category')
X['Gender'] = X['Gender'].astype('category')
X['Scholarship holder'] = X['Scholarship holder'].astype('category')
X['International'] = X['International'].astype('category')
X['Target'] = X['Target'].astype('category')
X.info()

### Информация о числовых признаках в нашей задаче:

In [ ]:
X.describe(include=['int64', 'float64'])

Здесь
- *min* – минимальное значение признака в таблице
- *max* – максимальное значение
- *mean* – среднее значение  
- *std* – среднеквадратическое отклонение (степень отклонения значений от среднего)  
- *25%* – 0.25-квантиль (25-й перцентиль, или первый квартиль) – значение, ниже которого лежит 25% значений  
- *50%* – медиана – значение, 50% значений меньше которого, а другие 50% больше
- *75%* – 0.75-квантиль (75-й перцентиль, или третий квартиль) – значение, ниже которого лежит около 75%, т.е. 75% значений меньше данного.

Балл студентов при поступлении (Admission grade) в среднем был ≈126.978 (а 95 и 190 – минимальный и максимальный баллы);  
25% студентов поступали с баллом не выше 117.9;  
75% студентов поступали с баллом не выше 134.8 (иначе говоря, лишь 25% студентов имеют балл выше);  
средний возраст на момент поступления – 23 года;  
25% студентов поступали в возрасте 19 лет и ранее;  
25% студентов поступали в возрасте выше 25 лет;  
средние баллы всех студентов за 1-й и 2-й семестры похожи.

Построим матрицу корелляции для числовых признаков:

In [ ]:
corrMatrix = X.select_dtypes(include=['int64', 'float64']).corr()
corrMatrix.style.background_gradient(cmap='coolwarm', vmin=-1, vmax=1)

Из данной матрицы можем заметить, что
- **Балл при поступлении (Admission grade) ощутимо коррелирует со средним баллом предыдущего образования (Previous qualification (grade))**
- **Существует существенная корреляция между средними баллами за 1-й и 2-й семестры обучения**
- Заметна некоторая корреляция между ВВП страны обучения и процентом безработицы.

### Информация о категориальных (в т.ч. бинарных) признаках в нашей задаче:

In [ ]:
X.describe(include=['category'])

Более всего одиноких студентов (1).  
Большинство студентов – дневной (1) формы обучения.  
Большинство студентов являются ранее получившими лишь среднее образование (1).  
Большинство студентов – португальцы (1).  
Матери большинства студентов имеют лишь среднее образование (12 лет обучения), а отцы большинства студентов имеют и вовсе лишь оконченный 1-й цикл базового образования (4-5 лет).  
Матери и отцы большинства студентов – неквалифицированные рабочие (9).  
Большинство студентов – женского пола (0).  
Примерно половина студентов – успешно окончившие обучение и получившие диплом.  
Число студентов-переселенцев немного больше числа не переселенцев.  
Число студентов с особенными образовательными потребностями очень мало.  
Стипендиатов около 25%.  
Международных студентов довольно мало, 110 человек (менее 3%).

# 2. Визуализация данных, борьба с выбросами

При помощи Pandas построим матрицу из диаграмм рассеивания.

In [ ]:
pd.plotting.scatter_matrix(X, figsize = (26, 26))
pass

### Избавление от выбросов
Для избавления от выбросов проанализируем средние баллы студентов при поступлении.

In [ ]:
X['Admission grade'].quantile([0.005,.01,.05,.1,.5,.9,.95,.99,.995])

Как мы видим, 99% студентов имеют средние баллы при поступлении от 99 до 166.831.

In [ ]:
X['Admission grade'].plot(kind='box')
pass

Исключим из рассмотрения 45 студентов, имеющих баллы при поступлении ниже 96.169 (слишком низкие) или выше 170 (слишком высокие).

In [ ]:
#outliers_1 = X[X['Admission grade'] < 96.169]
#outliers_2 = X[X['Admission grade'] > 170]
#outliers = pd.concat([outliers_1, outliers_2], ignore_index=True)
rowsToDrop = X[(X['Admission grade'] < X['Admission grade'].quantile(0.005)) | (X['Admission grade'] > X['Admission grade'].quantile(0.995))].index
Xclear = X.drop(rowsToDrop)

In [ ]:
Xclear['Admission grade'].plot(kind='box')
pass

Также заметим, что среди оставшихся студентов есть более 10% студентов, для которых средние баллы за 1-й и 2-й семестр почему-то равны 0.

In [ ]:
Xclear['Curricular units 1st sem (grade)'].plot(kind='box')
pass

Чтобы избавиться от этого странного явления, заменим все нулевые значения в столбцах *Curricular units 1st sem (grade)* и *Curricular units 2nd sem (grade)* на медианы соответствующих значений.